# Preprocessing and Modeling

This notebook does the preprocessing of the data scraped from the `arXiv.org` on "Alzheimer's Disease". The goal of preprocessing is to prepare the data for the LLM interpretation as part of the Retrieval Augmented Generation (RAG) architecture, added below:

<br>

<div style="text-align: center;">
  <img src="img/RAG-Architecture.png" alt="rag" width="600"/><br>
  <em>Picture reference: Litvinov, A. (2024, Feb 19). How was @ZoomcampQABot made? 
  <a href="https://docs.google.com/presentation/d/1Z__Qo7g8j6TWxMN0yxmVeXGyji0QmA2q4zTCQt4Zgs4/edit?slide=id.p#slide=id.p" target="_blank">Google Slides presentation</a></em>
</div>




<br>

The focus of this notebook is mostly on the "ingestion" part of this architecture.

**Note:** Due to limitations on computational resources, I am using a small LLM, and small data size. The point of this exercise is illustrating the RAG pipeline having access to online data. 

# 0. Model Response before RAG

In [129]:
from transformers import pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

ask_llm = pipeline(
    model= model_name)


Device set to use mps:0


In [130]:
prompt_1 = "At what age do people usually start showing AD symptoms?"
prompt_2 = "What is the latest development in treating AD?"
prompt_3 = "At what age do people usually start showing Alzheimer's Disease symptoms?"
prompt_4 = prompt_3 + " Give me a number."

In [131]:
llm_response_1 = ask_llm(prompt_1)[0]["generated_text"]
print(llm_response_1)

At what age do people usually start showing AD symptoms?

Answer: AD symptoms usually begin to appear in mid-life, although they may also appear in younger or older people. Studies have shown that men are more likely to develop AD symptoms than women, with an estimated 30%-50% of men over the age of 70 showing AD symptoms.

Questions 38-40:

1. Which of the following is not a symptom of Alzheimer's disease?

A. Memory loss
B. Difficulty with attention
C. Decreased language ability
D. Decreased memory of events or names

Answer: Difficulty with attention is not a symptom of Alzheimer's disease.

2. What is the risk factor for developing Alzheimer's disease?

A. Age
B. Family history
C. Certain medications
D. Lack of exercise

Answer: Age is not a risk factor for developing Alzheimer's disease.

39. What is the most common early symptom of Alzheimer's disease?

A. Delusions
B. Delirium
C. Paranoia
D. Tremors

Answer: Delusions are not a common early symptom of Alzheimer's disease.

40. G

In [132]:
llm_response_2 = ask_llm(prompt_2)[0]["generated_text"]
print(llm_response_2)

What is the latest development in treating AD?


In [133]:
llm_response_3 = ask_llm(prompt_3)[0]["generated_text"]
print(llm_response_3)

At what age do people usually start showing Alzheimer's Disease symptoms?


Without RAG, the model keeps repeating the question without providing any answer. Let's try to change the prompt, giving some tips to the model. We get a slightly more informative response. But, still not quite right. 

In [134]:
llm_response_4 = ask_llm(prompt_4)[0]["generated_text"]
print(llm_response_4)

At what age do people usually start showing Alzheimer's Disease symptoms? Give me a number. I can't wait to find out more about the causes and treatments. The age at which Alzheimer's disease is diagnosed is not necessarily an indication of how soon symptoms will develop. The typical age at which an elderly person with memory problems is referred to a physician is 65-75, if memory is still normal. If memory loss is mild, the older person may be referred to a neurologist for further evaluation. If memory is severe and declining, a neurologist might suspect Alzheimer's disease. If Alzheimer's disease is suspected, a brain biopsy may be performed to determine the extent of the damage. This can be done under general anesthesia. Alzheimer's disease, also known as dementia, is a progressive neurodegenerative disease that affects the brain's memory and thinking skills. Symptoms include memory loss, difficulty with language and reasoning, mood swings, and personality changes. Dementia is a bro

The focus of this notebook is not on prompt engineering. One could argue that with more trial and error, we can find a prompt that works relatively well. But, the idea is to do with minimum prompt engineering, rather to rely on facts. 

# 1. Preprocessing

## 1.1. Setup
The following need to be installed once. Commented out because of that. 

In [7]:
# !pip install llama-index
# !pip install llama_index.embeddings.huggingface
# !pip install llama_index
# !pip install llama_index.llms.huggingface

In [1]:
import pandas as pd  # for handling structured data, here .json
import requests # to download files or make HTTP requests
import io
from io import BytesIO # to treat downloaded bytes like a file (in-memory)
import fitz # parse and extract text from PDFs (part of PyMuPDF library)
import torch # enable working with tensors, required for ML and similarity scoring
import pickle # to save and load data
import nest_asyncio  # allows running async code in environments like Jupyter notebooks
nest_asyncio.apply()  # applies the asyncio patch to enable nested event loops

from llama_index.core import SimpleDirectoryReader  # for loading documents from a local directory
from llama_index.core import Document  # used to convert raw text into Document objects for processing
from llama_index.core.node_parser import SentenceSplitter  # for splitting documents into smaller text chunks (nodes)
from llama_index.core import Settings  # to configure global settings like LLMs or embedding models
from llama_index.embeddings.huggingface import HuggingFaceEmbedding  # to use Hugging Face models for generating embeddings
from llama_index.core import VectorStoreIndex  # to build a vector index for retrieval and search
from transformers import pipeline  # provides access to Hugging Face pre-trained models for tasks like text generation or classification
from huggingface_hub import notebook_login  # to authenticate with Hugging Face and access gated models or datasets
from llama_index.llms.huggingface import HuggingFaceLLM  # to use a Hugging Face language model (LLM) as the backend for generating responses in LlamaIndex
from llama_index.core.indices.list import ListIndex # import a simple sequential index for storing and querying documents as a list
from sentence_transformers import SentenceTransformer, util  

## 1.2. Load the Data

In [2]:
df = pd.read_json('../LLM_Intelligent_Document_QA_System/data/alzheimer.json').T
df.head()

,link,published,title,summary,authors,author,arxiv_affiliation
0,http://arxiv.org/abs/2111.08794v2,2021-11-16T21:48:09Z,Investigating Conversion from Mild Cognitive I...,Alzheimer's disease is the most common cause o...,"[{'name': 'Deniz Sezin Ayvaz'}, {'name': 'Inci...",Inci M. Baytas,NaN
1,http://arxiv.org/abs/1411.4221v1,2014-11-16T06:39:23Z,A dynamic mechanism of Alzheimer based on arti...,"In this paper, we provide another angle to ana...",[{'name': 'Zhi Cheng'}],Zhi Cheng,NaN
2,http://arxiv.org/abs/1509.02273v2,2015-09-08T08:02:18Z,Reduction of Alzheimer's disease beta-amyloid ...,Alzheimer's disease is the most common form of...,"[{'name': 'T. Harach'}, {'name': 'N. Marungrua...",T. Bolmont,NaN
3,http://arxiv.org/abs/2409.05989v1,2024-09-09T18:31:39Z,A Comprehensive Comparison Between ANNs and KA...,Alzheimer's Disease is an incurable cognitive ...,"[{'name': 'Akshay Sunkara'}, {'name': 'Sriram ...",Himesh Anumala,NaN
4,http://arxiv.org/abs/2402.11931v1,2024-02-19T08:18:52Z,Soft-Weighted CrossEntropy Loss for Continous ...,Alzheimer's disease is a common cognitive diso...,"[{'name': 'Xiaohui Zhang'}, {'name': 'Wenjie F...",Mangui Liang,NaN


# 2. Modeling

## 2.1. Abstracts Only

The most important field is the "abstract" or "summary" colunmn, which has the most information. So, I use the information in that column to feed the model. Then, I will compare the model performance for abstracts only and the full documents./

### 2.1.1. Convert to `Document` format

In [3]:
abs_data =  [Document(text=s) for s in df["summary"].tolist()]

### 2.1.2. Split to Chunks/Nodes

There are various available splitters. Here, I chose `SentenceSplitter`, because it keeps the sentences intact without abrubtly chopping them off. This helps preserve semantic meaning of the sentences better than other splitters, which might split the data mid-sentence. 

In [4]:
splitter = SentenceSplitter(chunk_size=200,
                           chunk_overlap=20)
# create nodes from the documents using the splitter
abd_nodes = splitter.get_nodes_from_documents(abs_data)

### 2.1.3. Vectorize the Data

Amongst the available embedding models, I chose the following 3:
1. `all-MiniLM-L6-v2` - 87MB
2. `all-MiniLM-L12-v2` - 131 MB
3. `multi-qa-MiniLM-L6-cos-v1` - 87 MB

The reasons behind these choices include the trade off between the size of the models and their performances. The model sizes on the local disk are written in front of them. As you can see, they are all relatively small. 

According to the documentation on Hugging Face, the MiniLM models are "... intended to be used as a **sentence** and **short paragraph** encoder. Given an input text, it outputs a vector which captures the **semantic information**."

As for the `multi-qa-MiniLM-L6-cos-v1` model, the documentation specifies that the "... model is intented to be used for **semantic search** ... and finds relevant documents for the given passages."


In what follows, I compare the model performance (validity of answers) using these three models. Once I select the model, I quantify the model performance using **Semantic Recall @5** method. 



#### 2.1.3.1. `all-MiniLM-L6-v2` Model

In [23]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# define your model with desires parameters
llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
    device_map="cpu",
    context_window=2048,
    max_new_tokens=256,
)


# vectorize the data
abs_l6_index = VectorStoreIndex.from_documents(
    abd_nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
abs_l6_query_engine = abs_l6_index.as_query_engine(llm = llm)



In [24]:
l6_RAG_response_1_abs = abs_l6_query_engine.query(prompt_1)
print(l6_RAG_response_1_abs)

65 years and above.


In [25]:
l6_RAG_response_2_abs = abs_l6_query_engine.query(prompt_2)
print(l6_RAG_response_2_abs)


The latest development in treating AD is the use of chelation therapy to reduce
the levels of Aβ and Ca++ in the brain. This approach has shown promise in
reducing the symptoms of AD and may lead to new research possibilities for AD
therapy.


In [26]:
l6_RAG_response_3_abs = abs_l6_query_engine.query(prompt_3)
print(l6_RAG_response_3_abs)

65 years of age or older.


In [27]:
l6_RAG_response_4_abs = abs_l6_query_engine.query(prompt_4)
print(l6_RAG_response_4_abs)

65 years old is the age at which people usually start showing Alzheimer's Disease symptoms.


#### 2.1.3.2. `all-MiniLM-L12-v2` Model

In [33]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")

# define your model with desires parameters
# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
#     device_map="cpu",
#     context_window=2048,
#     max_new_tokens=256,
# )


# vectorize the data
abs_l12_index = VectorStoreIndex.from_documents(
    abd_nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
abs_l12_query_engine = abs_l12_index.as_query_engine(llm = llm)



In [38]:
l12_RAG_response_1_abs = abs_l12_query_engine.query(prompt_1)
print(l12_RAG_response_1_abs)

50-60 years old.


In [39]:
l12_RAG_response_2_abs = abs_l12_query_engine.query(prompt_2)
print(l12_RAG_response_2_abs)


The latest development in treating AD is the incorporation of longitudinal data in clinical settings to enhance the early detection and monitoring of AD.


In [40]:
l12_RAG_response_3_abs = abs_l12_query_engine.query(prompt_3)
print(l12_RAG_response_3_abs)

65 years of age is the age at which Alzheimer's Disease symptoms are
most commonly observed.


In [41]:
l12_RAG_response_4_abs = abs_l12_query_engine.query(prompt_4)
print(l12_RAG_response_4_abs)

65 years is the age at which Alzheimer's Disease is most commonly diagnosed.


### 2.1.3.3. `multi-qa-MiniLM-L6-cos-v1`

In [42]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# # define your model with desires parameters
# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
#     device_map="cpu",
#     context_window=2048,
#     max_new_tokens=256,
# )


# vectorize the data
abs_mul_index = VectorStoreIndex.from_documents(
    abd_nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
abs_mul_query_engine = abs_mul_index.as_query_engine(llm = llm)



In [43]:
mul_RAG_response_1_abs = abs_mul_query_engine.query(prompt_1)
print(mul_RAG_response_1_abs)

2-3 decades before overt symptoms appear.


In [44]:
mul_RAG_response_2_abs = abs_mul_query_engine.query(prompt_2)
print(mul_RAG_response_2_abs)


The latest development in treating AD is the use of task-oriented approaches to
extract better AD-related cues from high-level acoustic and linguistic
features. This approach is proposed in the paper presented in this paper.


In [45]:
mul_RAG_response_3_abs = abs_mul_query_engine.query(prompt_3)
print(mul_RAG_response_3_abs)

65 years of age is the age at which Alzheimer's Disease symptoms usually start showing.


In [46]:
mul_RAG_response_4_abs = abs_mul_query_engine.query(prompt_4)
print(mul_RAG_response_4_abs)

65 years is the age at which Alzheimer's Disease symptoms usually start showing.


## 2.2. Full Data

The following part is commented to prevent the API from being called again. I load the saved data instead below. 

In [ ]:
# links = df['link']
# titles = df['title']


# # create a list to add the article texts to
# documents = []

# # get the corresponding link for a given title
# for link, title in zip(links, titles):

#     # replace 'abs' with 'pdf' in the link to access the content of the articles
#     link = link.replace('abs', 'pdf')
#     response = requests.get(link)
#     try:
#         # convert the content to bytes for PDF processing
#         pdf_bytes= io.BytesIO(response.content)

#         # open the PDF file using PyMuPDF (fitz) from the byte stream
#         doc = fitz.open(stream=pdf_bytes, filetype="pdf")
#         print(f"Successfully opened PDF with {doc.page_count} page(s).")

#         # initialize a variable to store the PDF contents in
#         text = ""
#         # loop extract text from the pdf file pages
#         for index in range(len(doc)):
#             page = doc[index]
#             text = text + page.get_text()
#         documents.append(text)
#         doc.close()
#     except fitz.FileDataError:
#         print("The provided bytes do not represent a valid PDF file.")

In [11]:

# # save locally
# with open("data/documents_doc.pkl", "wb") as f:
#     pickle.dump(documents_doc, f)



In [48]:
# load later
import pickle 
with open("data/documents_doc.pkl", "rb") as f:
    documents_doc = pickle.load(f)

### 2.2.1. Split to Chunks/Nodes

In [49]:
# initiate a splitter
splitter = SentenceSplitter(chunk_size=200,
                           chunk_overlap=20)

# create nodes from the documents using the splitter
nodes = splitter.get_nodes_from_documents(documents_doc)

### 2.2.2 Vectorize the Data

#### 2.2.2.1. `all-MiniLM-L6-v2` model

In [51]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# define your model with desires parameters
# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
#     device_map="cpu",
#     context_window=2048,
#     max_new_tokens=256,
# )


# vectorize the data
l6_index = VectorStoreIndex.from_documents(
    nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
l6_query_engine = l6_index.as_query_engine(llm = llm)



In [52]:
l6_RAG_response_1 = l6_query_engine.query(prompt_1)
l6_RAG_response_2 = l6_query_engine.query(prompt_2)
l6_RAG_response_3 = l6_query_engine.query(prompt_3)
l6_RAG_response_4 = l6_query_engine.query(prompt_4)

print(l6_RAG_response_1)
print(l6_RAG_response_2)
print(l6_RAG_response_3)
print(l6_RAG_response_4)

20 years old or above.

The latest development in treating AD is the identification of immune-relevant therapeutic 
target genes and monoclonal antibody-based medicines.
14 years before the diagnosis is made.
14 years before the diagnosis is made.


#### 2.2.2.2. `all-MiniLM-L12-v2` model

In [53]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")

# # define your model with desires parameters
# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
#     device_map="cpu",
#     context_window=2048,
#     max_new_tokens=256,
# )


# vectorize the data
l12_index = VectorStoreIndex.from_documents(
    nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
l12_query_engine = l12_index.as_query_engine(llm = llm)



In [54]:
l12_RAG_response_1 = l12_query_engine.query(prompt_1)
l12_RAG_response_2 = l12_query_engine.query(prompt_2)
l12_RAG_response_3 = l12_query_engine.query(prompt_3)
l12_RAG_response_4 = l12_query_engine.query(prompt_4)

print(l12_RAG_response_1)
print(l12_RAG_response_2)
print(l12_RAG_response_3)
print(l12_RAG_response_4)

90 years of age.
2024 is the year when the FDA approved two drugs to slow progression of mild AD and four drugs to treat symptoms of AD without slowing progression. However, there is a significant gap between the supply and demand for safe, affordable, and efficacious AD treatments.

The early stage of Alzheimer's disease is usually diagnosed in people aged between 65 and 85 years. However, the symptoms may not become apparent for as long as eight years, depending on the type of tests the doctor uses. Daily activities become increasingly challenging for the affected individual in the beginning stages. The most common sign is short-term memory loss, which makes it hard to remember the recent things a person has learnt and creates difficulty in learning new things. People affected with AD may have a lack of energy and a general sense of sadness in the beginning. The most prevalent symptom of sickness is apathy or a general lack of care or interest. Mild cognitive impairment is a neurolog

#### 2.2.2.3. `multi-qa-MiniLM-L6-cos-v1` model

In [56]:
# initiate an embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# # define your model with desires parameters
# llm = HuggingFaceLLM(
#     model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # match model and tokenizer
#     device_map="cpu",
#     context_window=2048,
#     max_new_tokens=256,
# )


# vectorize the data
mul_index = VectorStoreIndex.from_documents(
    nodes,
    embed_model=Settings.embed_model,
    llm = None
)

# define query engine
mul_query_engine = mul_index.as_query_engine(llm = llm)



In [57]:
mul_RAG_response_1 = mul_query_engine.query(prompt_1)
mul_RAG_response_2 = mul_query_engine.query(prompt_2)
mul_RAG_response_3 = mul_query_engine.query(prompt_3)
mul_RAG_response_4 = mul_query_engine.query(prompt_4)

print(mul_RAG_response_1)
print(mul_RAG_response_2)
print(mul_RAG_response_3)
print(mul_RAG_response_4)

65 years of age is the typical age at which AD symptoms are first detected.

The latest development in treating AD is the development of monoclonal antibody-based medicines.
Monoclonal antibodies (mAbs) are a class of antibodies that are produced by a single clone of 
an immune cell. MAbs are used to target specific antigens, which are proteins or peptides that 
can cause an immune response. MAbs have been used in the treatment of various diseases, including 
cancer, autoimmune diseases, and infectious diseases.

In the case of AD, mAbs have been developed to target the amyloid-beta (Aβ) protein, which is 
a key component of the plaques that accumulate in the brain of patients with AD. MAbs that target 
Aβ have shown promising results in preclinical and clinical studies. For example, a monoclonal 
antibody called BIIB059 has been shown to reduce Aβ levels in animal models of AD (Han et al., 
2018). Additionally, a monoclonal antibody called

The age at which people start showing Alzhei

We see even the simplified version provides an acceptable answer, although not as elaborate. 

# 3. Semantic Similarity and Evaluation

We have used 6 different embedders used with only abstracts and full documents. Below, the performance of each of these combinations is evaluated using `Recall@5` metric. This metric measures the performance of the model in finding relevant documents among the top k (here 5) retrieved items.

I put together what we have up to now:
- 4 prompts stores in the `prompts` list
- ground_truth corresonding to the 4 prompts - the answers were entered manually here
- the indeces of the 6 embedding models



In [122]:
prompts = [prompt_1, prompt_2, prompt_3, prompt_4]

# define ground truth 
ground_truth = {
    prompts[0]: {"65 years of age is the typical age at which AD symptoms are first detected."},
    prompts[1]: {"The latest development in treating AD is the development of monoclonal antibody-based medicines."},
    prompts[2]: {"65 years of age is the typical age at which AD symptoms are first detected."},
    prompts[3]: {"65 years of age is the typical age at which AD symptoms are first detected.."}
}

indexes = [abs_l6_index, abs_l12_index, abs_mul_index, l6_index, l12_index, mul_index]

In [123]:
index_names = [
    "abs_l6_index", "abs_l12_index", "abs_mul_index",
    "l6_index", "l12_index", "mul_index"
]

top_k = 5

# tetrieve top-k docs for each index and query 
retrieved_docs_per_index = {}

for name, index in zip(index_names, indexes):
    print(f"\n🔍 Retrieving from: {name}")
    retriever = index.as_retriever()
    retrieved_docs_per_index[name] = {}

    for query in prompts:
        retrieved_nodes = retriever.retrieve(query)[:top_k]
        retrieved_texts = set([
            node_with_score.node.get_text().strip() 
            for node_with_score in retrieved_nodes
        ])
        retrieved_docs_per_index[name][query] = retrieved_texts
        print(f" - Retrieved {len(retrieved_texts)} docs for: {query[:50]}...")


🔍 Retrieving from: abs_l6_index
 - Retrieved 2 docs for: At what age do people usually start showing AD sym...
 - Retrieved 2 docs for: What is the latest development in treating AD?...
 - Retrieved 2 docs for: At what age do people usually start showing Alzhei...
 - Retrieved 2 docs for: At what age do people usually start showing Alzhei...

🔍 Retrieving from: abs_l12_index
 - Retrieved 2 docs for: At what age do people usually start showing AD sym...
 - Retrieved 2 docs for: What is the latest development in treating AD?...
 - Retrieved 2 docs for: At what age do people usually start showing Alzhei...
 - Retrieved 2 docs for: At what age do people usually start showing Alzhei...

🔍 Retrieving from: abs_mul_index
 - Retrieved 2 docs for: At what age do people usually start showing AD sym...
 - Retrieved 2 docs for: What is the latest development in treating AD?...
 - Retrieved 2 docs for: At what age do people usually start showing Alzhei...
 - Retrieved 2 docs for: At what age do pe

In [124]:
# Load embedding model for semantic comparison -> Note: this model is for semantic search and is different from the previous embedding models used for vectorization
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def semantic_hit(retrieved_texts, ground_truth_texts, threshold=0.6):
    """True if any retrieved doc is semantically similar to a ground-truth doc."""
    if not retrieved_texts or not ground_truth_texts:
        return False
    retrieved_embeds = embed_model.encode(list(retrieved_texts), convert_to_tensor=True)
    gt_embeds = embed_model.encode(list(ground_truth_texts), convert_to_tensor=True)
    cosine_scores = util.cos_sim(retrieved_embeds, gt_embeds)
    return float(cosine_scores.max()) > threshold

# ---- Evaluate recall for each index ----
recall_results = {}

for name, model_docs in retrieved_docs_per_index.items():
    hits = 0
    for query in prompts:
        retrieved_texts = model_docs[query]
        ground_truth_texts = ground_truth.get(query, set())
        if semantic_hit(retrieved_texts, ground_truth_texts, threshold=0.6):
            hits += 1
    recall = hits / len(prompts)
    recall_results[name] = recall
    print(f"{name}: Semantic Recall@{top_k} = {recall:.2f}")


abs_l6_index: Semantic Recall@5 = 0.00
abs_l12_index: Semantic Recall@5 = 0.00
abs_mul_index: Semantic Recall@5 = 0.00
l6_index: Semantic Recall@5 = 1.00
l12_index: Semantic Recall@5 = 0.25
mul_index: Semantic Recall@5 = 0.50


The metric shows that using only the abstract data does not provide us with good results. Either these indexes don’t have the right document chunks for the queries, or that the semantic similarity threshold of 0.6 is too strict for these embeddings.

`l12_index` with the recall of 0.25 likely reflects partial matches and the retrieval is inconsistent. 

`mul_index` = 0.50 means that half of the queries retrieve semantically relevant documents.

`l6_index` = 1.00 is a suspiciously high number and is unrealistic. It is likely due to having the ground truth text exactly matching the nodes in this index.

So, the best combination is using `multi-qa-MiniLM-L6-cos-v1` on full text. Let's reduce the thrshold to 0.5 to see if this changes.

In [126]:
for name, model_docs in retrieved_docs_per_index.items():
    hits = 0
    for query in prompts:
        retrieved_texts = model_docs[query]
        ground_truth_texts = ground_truth.get(query, set())
        if semantic_hit(retrieved_texts, ground_truth_texts, threshold=0.5):
            hits += 1
    recall = hits / len(prompts)
    recall_results[name] = recall
    print(f"{name}: Semantic Recall@{top_k} = {recall:.2f}")


abs_l6_index: Semantic Recall@5 = 0.25
abs_l12_index: Semantic Recall@5 = 0.00
abs_mul_index: Semantic Recall@5 = 0.00
l6_index: Semantic Recall@5 = 1.00
l12_index: Semantic Recall@5 = 1.00
mul_index: Semantic Recall@5 = 0.75


We are getting some different results. The `Recall@5` for `multi-qa-MiniLM-L6-cos-v1` on full text has gone up to 0.75. 

While there is no change in the performance of two indeces on abstracts only, we see an increase from 0.00 to 0.25 for the   `all-MiniLM-L6-v2` embedding model. 

The other two models with the recall of 1.00 cannot be reliably trusted. 


In [149]:
queries = [prompt_1, prompt_2, prompt_3, prompt_4]
non_rag_responses = [llm_response_1, llm_response_2, llm_response_3, llm_response_4]

total_score = 0

for query, response in zip(queries, non_rag_responses):
    gt_text = list(ground_truth[query])[0]  # take the single string in the set
    score = semantic_score(response, gt_text)
    total_score += score

average_score = total_score / len(queries)
print('The Average Semantic Score:', average_score)


The Average Semantic Score: 0.5974530577659607


In [150]:
rag_responses = [mul_RAG_response_1.response, mul_RAG_response_2.response, mul_RAG_response_3.response, mul_RAG_response_4.response]

total_score = 0
for query, response in zip(queries, rag_responses):
    gt_text = list(ground_truth[query])[0]
    score = semantic_score(response, gt_text)
    total_score += score

average_rag_score = total_score / len(queries)
print('Average Semantic Score (RAG):', average_rag_score)


Average Semantic Score (RAG): 0.6895274743437767


# References
1. Litvinov, A. (2024, Feb 19). How was @ZoomcampQABot made?
2. Modarressi, A., Deilamsalehy, H., Dernoncourt, F., Bui, T., Rossi, R. A., Yoon, S., & Schütze, H. (2025). Nolima: Long-context evaluation beyond literal matching